In [15]:
import PIL
from torchvision import transforms

def get_test_img(file):
    test_img = PIL.Image.open(f'test pictures\\{file}')
    return transforms.ToTensor()(test_img)

In [11]:
import torch
import os
from models.SRDIFFBuilder import SRDiffBuilder
from utils.model_utils import load_model

saved_dir = 'C:\\Users\\adria\\Desktop\\TFG-code\\SR-model-benchmarking\\final_models\\SRDiff'
models_files = os.listdir(saved_dir)
device = torch.device(0)
model_builder = SRDiffBuilder()
model_builder = model_builder.set_standart()
sr_images = []
for model_file, timesteps in zip(models_files[:-1], (200, 300) ):
    try:
        torch.cuda.empty_cache()
        bicubic = get_test_img("Amnesty POI-6-3-1.png").unsqueeze(0).to(device)
        lr = get_test_img("Amnesty POI-6-3-1-lr.png").unsqueeze(0).to(device)    
        model_builder = model_builder.set_timesteps(timesteps)
        model, _ = model_builder.build()
        model.to(device)
        model = load_model(model, model_file, saved_dir)
        
        sr, _ = model.sample(lr, bicubic, (1, 3, 256,256), use_rrdb=True)
        torch.cuda.empty_cache()
        sr_images.append(sr.squeeze().to('cpu').detach())
    except Exception as e:
        print(e)
        continue

The model weights have been loaded from 'C:\Users\adria\Desktop\TFG-code\SR-model-benchmarking\final_models\SRDiff\SRDiff_200steps_300epochs.pt'
The model weights have been loaded from 'C:\Users\adria\Desktop\TFG-code\SR-model-benchmarking\final_models\SRDiff\SRDiff_300steps_100epochs.pt'


In [13]:
from utils.tensor_utils import tensor2img

for idx, tensor in enumerate(sr_images):
    img_np = tensor2img(tensor)
    img = PIL.Image.fromarray(img_np)
    img.save(f"test pictures\\SRDiff\\{models_files[idx]}.png")

In [21]:
from models.SR32Builder import SR32Builder 

model_dir = 'C:\\Users\\adria\\Desktop\\TFG-code\\SR-model-benchmarking\\final_models\\SR3\\SR3+ Epoch50.pt'
model_builder = SR32Builder()
model_builder = model_builder.set_plus()
model = model_builder.build()
model.to(device)

bicubic = get_test_img("Amnesty POI-6-3-1.png").unsqueeze(0).to(device)
lr = get_test_img("Amnesty POI-6-3-1-lr.png").unsqueeze(0).to(device)

sr = model.super_resolution(bicubic)

sampling loop time step: 100%|██████████| 2000/2000 [00:38<00:00, 52.32it/s]


In [22]:
img_np = tensor2img(sr)
img = PIL.Image.fromarray(img_np)
img.save(f"test pictures\\SR3_plus.png")